In [2]:
import pandas as pd
import math as mt
from sklearn.preprocessing import LabelEncoder

data = pd.read_csv('Wong.csv', delimiter=';', decimal=',')
#encode fitur tipe biner
X = data.iloc[:,:].values
labelEncode_X = LabelEncoder()
X[:,4] = labelEncode_X.fit_transform(X[:,4])
df = pd.DataFrame(data)
df.style.highlight_null(null_color='red').hide_index()

no,id,days,duration,sex,age,piq,viq
1,3358,30,4,Male,20.6708,87,89
2,3535,16,17,Male,55.2882,95,77
3,3547,40,1,Male,55.9151,95,116
4,3592,13,10,Male,61.6646,59,73
5,3728,19,6,Male,30.1273,67,73
6,3790,13,3,Male,57.0623,76,69
7,3807,37,5,Male,24.6762,74,77
8,3808,31,7,Male,28.2683,91,110
9,4253,40,3,Male,22.6037,115,110
10,4356,31,7,Male,21.399,86,83


In [3]:
def Zscore(x,mean,std):
    top = x - mean
    if top==0:
        return top
    else:
        return round(top / std, 2)

#menghitung jarak tipe numerikal
def euclidianDistance(x,y):
    dis = 0
    for i in range(len(x)):
        dis += (x[i] - y[i]) ** 2
    return round(mt.sqrt(dis),2)

#Menghitung jarak tipe binary
def distanceSimetris(x,y):
    q=r=s=t=0
    for i in range(len(x)):
        if x[i]==1 and y[i]==1:
            q+=1
        elif x[i]==1 and y[i]==0:
            r+=1
        elif x[i]==0 and y[i]==1:
            s+=1
        elif x[i]==0 and y[i]==0:
            t+=1
    return ((r+s)/(q+r+s+t))

def normalisasi(num, col_x): 
    return Zscore(num, pd.Series(data[col_x].values).mean(), pd.Series(data[col_x].values).std())

In [4]:
c_j = 0
for j in df['age'].isna():
    if j == True:
        col_missing = c_j
    c_j+=1

In [5]:
missing_data = df.iloc[col_missing, [2,3,6,7]].values
missing_normal = [normalisasi(missing_data[0],data.columns[2]), normalisasi(missing_data[1],data.columns[3]), normalisasi(missing_data[2],data.columns[6]), normalisasi(missing_data[3],data.columns[7])]

for i in range(len(data[data.columns[0]])):
    if i==col_missing:
        continue;
    select_data = df.iloc[i, [2,3,6,7]].values
    normal_data = [normalisasi(select_data[0],data.columns[2]), normalisasi(select_data[1],data.columns[3]), normalisasi(select_data[2],data.columns[6]), normalisasi(select_data[3],data.columns[7])]
    data.loc[i, 'jarak'] =  euclidianDistance(missing_normal,normal_data) + distanceSimetris([X[col_missing, 4]],[X[i, 4]])

In [6]:
df = pd.DataFrame(data)
df.sort_values(by='jarak', axis=0, ascending=True, inplace=True) 
df.iloc[-1, [5]] = round(df.iloc[0:2,5].mean(), 2)
df.style.hide_index()

no,id,days,duration,sex,age,piq,viq,jarak
229,5397,328,0,Female,62.7981,121,108,0.25
13,4705,18,1,Female,21.6838,127,109,0.53
205,6228,174,3,Female,31.5537,114,108,0.6
284,6228,662,3,Female,31.5537,128,111,0.64
161,5699,138,1,Female,34.2231,110,107,0.84
153,4705,146,1,Female,21.6838,133,111,0.86
196,5208,193,8,Female,21.3771,133,111,0.88
300,5837,962,1,Female,33.3087,109,110,1.07
139,6253,128,0,Female,46.4038,104,112,1.31
141,6665,119,3,Female,23.0171,106,94,1.33
